In [1]:
import chardet
import numpy as np
import pandas as pd
import datetime
import codecs

import progressbar
from time import sleep

import matplotlib.pyplot as plt

In [39]:
filename = './data/reservations2017.csv'

In [46]:
# attempt to load the data using various codecs
types_of_encoding = ["utf8", "cp1252"]
for encoding_type in types_of_encoding:
    with codecs.open(filename, encoding = encoding_type, errors ='replace') as csvfile:
        rez = pd.read_csv(csvfile)

In [47]:
# function to format the imported data, drop all categories of reservations except campsites
def nps_site_format(df):
    # get only reservations from the National Park Service, OrdID of 128
    df = df[df['OrgID'] == 128]
    
    # get only site type entities
    df = df[df['EntityType'] == 'Site']
    
    # convert date columns to datetime, y-m-d
    df['EndDate'] = pd.to_datetime(df['EndDate'], format= '%Y-%m-%d')
    df['StartDate'] = pd.to_datetime(df['StartDate'], format= '%Y-%m-%d')
    df['OrderDate'] = pd.to_datetime(df['OrderDate'], format= '%Y-%m-%d')
    
    # calculate the stay length of the reservation
    df['StayLen'] = df['EndDate'] - df['StartDate']
    df['StayLen'] = df['StayLen'].dt.days + 1
    
    # calculate the days in advance the resveration was made
    # note: some reservations were recorded after the start date, those values are imputed as zero
    df['BookingHorizon'] = df['StartDate'] - df['OrderDate']
    df['BookingHorizon'] = df['BookingHorizon'].dt.days.clip(lower=0)
    
    # calculate the daily rate for each reservation
    df['DailyRate'] = df['TotalBeforeTax'] / df['StayLen']
    
    # fill nulls in fee columns with zeroes
    df[['UseFee', 'TranFee', 'AttrFee']] = df[['UseFee', 'TranFee', 'AttrFee']].fillna(0)
    
    # drop facility attribute columns
    df = df.drop(axis = 1, columns = ['FacilityState', 'FacilityLongitude', 'FacilityLatitude', 
                                      'CustomerZIP', 'CustomerState', 'CustomerCountry', 'FacilityZIP', 'EntityID'])
    
    # drop categorical columns
    df = df.drop(axis = 1, columns = ['Tent', 'Popup', 'Trailer', 
                              'RVMotorhome', 'Boat', 'HorseTrailer', 'Car', 'FifthWheel', 
                              'Van', 'CanoeKayak', 'BoatTrailer', 'Motorcycle', 'Truck', 
                              'Bus', 'Bicycle', 'Snowmobile', 'OffRoadlAllTerrainVehicle', 
                              'PowerBoat', 'PickupCamper', 'LargeTentOver9x12', 'SmallTent', 'Marinaboat'])
    
    # drop nulls in important columns
    df = df.dropna(axis = 0, subset = ['FacilityID', 'StartDate', 'EndDate'])
    
    # drop duplicate order numbers
    df = df.drop_duplicates(['OrderNumber'])
    
    return df

In [48]:
# format imported data
rez = nps_site_format(rez)

In [52]:
rez['RegionDescription'].unique()

array(['Grand Canyon National Park', 'Pinnacles National Park',
       'Death Valley National Park', 'Joshua Tree National Park',
       'Gulf Islands National Seashore', 'Big Cypress National Preserve',
       'Acadia National Park', 'Yosemite National Park',
       'Point Reyes National Seashore', 'Colorado National Monument',
       'Great Smoky Mountains National Park',
       'Rocky Mountain National Park', 'Olympic National Park',
       'Big Bend National Park', 'Channel Islands National Park',
       'Shenandoah National Park', 'Greenbelt Park',
       'Catoctin National Park', 'Cape Hatteras National Seashore',
       'Zion National Park', 'Chaco Culture National Historic Park',
       'Sequoia and Kings Canyon National Park',
       'Everglades National Park', 'Whiskeytown National Recreation Area',
       'Gateway National Recreation Area',
       'Golden Gate National Recreation Area', 'Blue Ridge Parkway',
       'Assateague Island National Seashore',
       'Big South For

In [53]:
rez.describe()

,HistoricalReservationID,OrgID,ParentLocationID,LegacyFacilityID,ProductID,FacilityID,Tax,UseFee,TranFee,AttrFee,TotalBeforeTax,TotalPaid,NumberOfPeople,StayLen,BookingHorizon,DailyRate
count,6.004880e+05,600488.0,600488.000000,600488.000000,600488.000000,600488.000000,600488.000000,600488.000000,600488.000000,600488.000000,600488.000000,600488.000000,600488.000000,600488.000000,600488.000000,600488.000000
mean,2.623824e+09,128.0,74315.426185,76775.493650,258476.953907,234746.094625,0.079660,39.498325,2.096913,0.278234,42.569217,42.649804,4.282828,3.225808,72.963336,13.085198
std,1.152103e+08,0.0,21.803027,15392.618806,81176.864421,5634.466521,1.055896,50.489793,5.089565,3.720690,49.651435,49.818394,8.484398,1.694500,68.948004,15.347127
min,2.434323e+09,128.0,74265.000000,70852.000000,139983.000000,232433.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
25%,2.523479e+09,128.0,74324.000000,70940.000000,203241.000000,232461.000000,0.000000,16.000000,0.000000,0.000000,18.000000,18.000000,2.000000,2.000000,9.000000,8.000000
50%,2.618144e+09,128.0,74325.000000,70971.000000,206798.000000,232490.000000,0.000000,26.000000,0.000000,0.000000,30.000000,30.000000,3.000000,3.000000,51.000000,12.000000
75%,2.721671e+09,128.0,74327.000000,72479.000000,304300.000000,234035.000000,0.000000,52.000000,0.000000,0.000000,52.000000,52.000000,5.000000,4.000000,133.000000,15.333333
max,2.830804e+09,128.0,74330.000000,152990.000000,460371.000000,261700.000000,84.920000,3920.000000,420.000000,196.000000,3920.000000,3920.000000,600.000000,39.000000,719.000000,800.000000


In [54]:
rez.isnull().sum()

HistoricalReservationID    0
OrderNumber                0
Agency                     0
OrgID                      0
CodeHierarchy              0
RegionCode                 0
RegionDescription          0
ParentLocationID           0
ParentLocation             0
LegacyFacilityID           0
Park                       0
SiteType                   0
UseType                    0
ProductID                  0
EntityType                 0
FacilityID                 0
Tax                        0
UseFee                     0
TranFee                    0
AttrFee                    0
TotalBeforeTax             0
TotalPaid                  0
StartDate                  0
EndDate                    0
OrderDate                  0
NumberOfPeople             0
StayLen                    0
BookingHorizon             0
DailyRate                  0
dtype: int64

In [55]:
rez.dtypes

HistoricalReservationID             int64
OrderNumber                        object
Agency                             object
OrgID                               int64
CodeHierarchy                      object
RegionCode                         object
RegionDescription                  object
ParentLocationID                    int64
ParentLocation                     object
LegacyFacilityID                    int64
Park                               object
SiteType                           object
UseType                            object
ProductID                           int64
EntityType                         object
FacilityID                        float64
Tax                               float64
UseFee                            float64
TranFee                           float64
AttrFee                           float64
TotalBeforeTax                    float64
TotalPaid                         float64
StartDate                  datetime64[ns]
EndDate                    datetim

In [56]:
for dtype in ['float','int','object']:
    selected_dtype = rez.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))

Average memory usage for float columns: 4.58 MB
Average memory usage for int columns: 4.58 MB
Average memory usage for object columns: 37.72 MB


In [57]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [58]:
rez_int = rez.select_dtypes(include=['int'])
converted_int = rez_int.apply(pd.to_numeric,downcast='unsigned')
print(mem_usage(rez_int))
print(mem_usage(converted_int))
compare_ints = pd.concat([rez_int.dtypes, converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

32.07 MB
15.46 MB


,before,after
uint8,NaN,1.0
uint16,NaN,1.0
uint32,NaN,4.0
int64,6.0,NaN


In [59]:
rez_float = rez.select_dtypes(include=['float'])
converted_float = rez_float.apply(pd.to_numeric,downcast='float')
print(mem_usage(rez_float))
print(mem_usage(converted_float))
compare_floats = pd.concat([rez_float.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['before','after']
compare_floats.apply(pd.Series.value_counts)

50.39 MB
27.49 MB


,before,after
float32,NaN,10.0
float64,10.0,NaN


In [60]:
optimized_rez = rez.copy()
optimized_rez[converted_int.columns] = converted_int
optimized_rez[converted_float.columns] = converted_float
print(mem_usage(rez))
print(mem_usage(optimized_rez))

502.00 MB
462.48 MB


In [61]:
rez.to_csv("./Data/Cleaned-Data/nps_res_2017.csv")
optimized_rez.to_csv("./Data/Cleaned-Data/nps_optimized_2017.csv")

In [62]:
optimized_rez.head()

,HistoricalReservationID,OrderNumber,Agency,OrgID,CodeHierarchy,RegionCode,RegionDescription,ParentLocationID,ParentLocation,LegacyFacilityID,...,AttrFee,TotalBeforeTax,TotalPaid,StartDate,EndDate,OrderDate,NumberOfPeople,StayLen,BookingHorizon,DailyRate
4,2434322626,2-36683841,NPS,128,|1|70904|74324|74282|70971|,GRCA-8210,Grand Canyon National Park,74324,Intermountain Region,70971,...,0.0,18.0,18.0,2016-10-16,2016-10-17,2016-10-01,4,2.0,15.0,9.000000
5,2434322664,2-36683842,NPS,128,|1|70904|74327|73983|73984|,PINN-8450,Pinnacles National Park,74327,Pacific West Region,73984,...,0.0,46.0,46.0,2016-11-25,2016-11-27,2016-10-01,4,3.0,55.0,15.333333
6,2434322789,2-36683844,NPS,128,|1|70904|74324|74282|70971|,GRCA-8210,Grand Canyon National Park,74324,Intermountain Region,70971,...,0.0,18.0,18.0,2016-10-04,2016-10-05,2016-10-01,2,2.0,3.0,9.000000
11,2434323612,2-36683171,NPS,128,|1|70904|74327|74277|70978|,DEVA-8130,Death Valley National Park,74327,Pacific West Region,70978,...,0.0,44.0,44.0,2016-10-16,2016-10-18,2016-10-01,2,3.0,15.0,14.666667
13,2434324034,2-36683173,NPS,128,|1|70904|74327|74286|70952|,JOTR-8330,Joshua Tree National Park,74327,Pacific West Region,70952,...,0.0,10.0,10.0,2017-03-10,2017-03-12,2016-10-01,5,3.0,160.0,3.333333


## Testing Fields
---

In [63]:
datelist = pd.date_range(rez['StartDate'].min(), rez['EndDate'].max()).tolist()
print(datelist[94].date())
jan_1 = datelist[500]
jan_1_filter = (rez['StartDate'] <= jan_1) & (rez['EndDate'] >= jan_1)

2017-01-01


In [64]:
day = optimized_rez[jan_1_filter].groupby('FacilityID').agg({'StayLen': np.mean, 'BookingHorizon': np.mean,
                                                       'UseFee' : np.mean, 'NumberOfPeople' : np.sum, 
                                                       'DailyRate' : np.sum})

In [65]:
widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage(),
               ' ', progressbar.ETA(),
               ' ', progressbar.AdaptiveETA()]

In [66]:
date_park_list = []
test_len = optimized_rez[jan_1_filter].groupby(['Park', 'RegionDescription']).mean().shape[0]
bar = progressbar.ProgressBar(maxval=test_len, widgets=widgets)

bar.start()
i = 0

for index, row in optimized_rez[jan_1_filter].groupby(['Park', 'RegionDescription', 'FacilityID']).nunique().iterrows():
    
    day_group = optimized_rez[jan_1_filter].groupby('FacilityID').agg({'StayLen': np.mean, 'BookingHorizon': np.mean,
                                                       'UseFee' : np.mean, 'NumberOfPeople' : np.sum, 
                                                       'DailyRate' : np.sum})
    
    stay_len = day_group.loc[index[2], 'StayLen']
    book_hor = day_group.loc[index[2], 'BookingHorizon']
    avg_fee = day_group.loc[index[2], 'UseFee']
    num_people = day_group.loc[index[2], 'NumberOfPeople']
    day_revenue = day_group.loc[index[2], 'DailyRate']
    
    
    date_park_list.append([jan_1, index[2], index[0], index[1], row['OrderNumber'], num_people, stay_len, book_hor, avg_fee, day_revenue])
    sleep(0.00001)
    
    i += 1
    bar.update(i)
bar.finish()

[============================================] 100% Time: 0:00:00 Time: 0:00:00


In [67]:
rez_test = pd.DataFrame(date_park_list, columns = ['Date', 'FacilityID', 'Site', 
                                        'Park', 'Reservations', 'NumberOfPeople', 
                                        'AvgStayLen', 'AvgBookingHorizon', 'AverageFee', 'DailyRevenue'])

rez_test

,Date,FacilityID,Site,Park,Reservations,NumberOfPeople,AvgStayLen,AvgBookingHorizon,AverageFee,DailyRevenue
0,2018-02-11,232473.0,BLACK ROCK CAMPGROUND,Joshua Tree National Park,16,48,5.875000,163.375000,38.125000,152.095230
1,2018-02-11,250901.0,Bonita Canyon Campground,Chiricahua National Monument,1,2,6.000000,184.000000,30.000000,5.000000
2,2018-02-11,246890.0,Burns Lake Campground,Big Cypress National Preserve,1,8,3.000000,160.000000,0.000000,3.333333
3,2018-02-11,234038.0,CHISOS BASIN (BIG BEND),Big Bend National Park,4,15,5.000000,164.750000,31.500000,34.283333
4,2018-02-11,234013.0,CHISOS BASIN GROUP CAMPGROUND,Big Bend National Park,1,12,5.000000,184.000000,0.000000,2.000000
5,2018-02-11,233381.0,CIRCLE X RANCH GROUP CAMPGROUND,Santa Monica Mountains National Recreation Area,1,10,2.000000,184.000000,35.000000,17.500000
6,2018-02-11,232471.0,COTTONWOOD GROUP,Joshua Tree National Park,3,60,3.000000,304.333344,50.000000,53.333332
7,2018-02-11,250796.0,Davis Bayou Campground,Gulf Islands National Seashore,20,42,10.600000,162.250000,77.550003,159.090866
8,2018-02-11,232495.0,FLAMINGO,Everglades National Park,35,77,10.257143,172.514282,0.000000,0.000000
9,2018-02-11,234704.0,FORT PICKENS CAMPGROUND,Gulf Islands National Seashore,72,170,12.722222,172.875000,106.805557,641.042175


## Site Aggregation for Modeling
---

In [68]:
def nps_site_aggregator_alternate(df):
    
    # establish list for each site's daily stats
    site_list = []
    
    # create list of days to run through
    datelist = pd.date_range(df['StartDate'].min(), df['EndDate'].max()).tolist()
    
    # create progress bar object
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage(),
               ' ', progressbar.ETA(),
               ' ', progressbar.AdaptiveETA()]
    
    bar = progressbar.ProgressBar(maxval=len(datelist), widgets=widgets)
    
    bar.start()
    count = 0
    
    # cycle through all days
    for date in datelist:
        
        #create a mask to find only reservations that inlcude the specified date
        date_mask = (df['StartDate'] <= date) & (df['EndDate'] >= date)
        
        # cycle through all resvervations, as grouped by the site and the park
        for index, row in df[date_mask].groupby(['Park', 'RegionDescription', 'FacilityID']).nunique().iterrows():
            
           # get the average stay length, booking horizon, and use fee for the reservations for the date
            stay_len = df[date_mask].groupby(['FacilityID']).mean().loc[index[2], 'StayLen']
            book_hor = df[date_mask].groupby(['FacilityID']).mean().loc[index[2], 'BookingHorizon']
            avgfee = df[date_mask].groupby(['FacilityID']).mean().loc[index[2], 'UseFee']
            
            # get the total number of people per the grouped reservations
            num_people = df[date_mask].groupby(['FacilityID']).sum().loc[index[2], 'NumberOfPeople']
            
            # get the daily revenue from the daily rate for each reservation
            day_revenue = df[date_mask].groupby(['FacilityID']).sum().loc[index[2], 'DailyRate']
            
            # add daily reservation information to the list
            site_list.append([date.date(), index[2], index[0], index[1], row['OrderNumber'], num_people, stay_len, book_hor, avgfee, day_revenue])
            
        
        count += 1
        bar.update(count)
    
    bar.finish()
    return pd.DataFrame(site_list, columns = ['Date', 'FacilityID', 'Site', 'Park', 'Reservations', 
                                              'NumberOfPeople', 'AvgStayLen', 'AvgBookingHorizon', 'AverageFee', 'DailyRevenue'])

In [69]:
def nps_site_aggregator(df):
    
    # establish list for each site's daily stats
    site_list = []
    
    # create list of days to run through
    datelist = pd.date_range(df['StartDate'].min(), df['EndDate'].max()).tolist()
    
    # create progress bar object
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage(),
               ' ', progressbar.ETA(),
               ' ', progressbar.AdaptiveETA()]
    
    bar = progressbar.ProgressBar(maxval=len(datelist), widgets=widgets)
    
    bar.start()
    count = 0
    
    # cycle through all days
    for date in datelist:
        
        #create a mask to find only reservations that inlcude the specified date
        date_mask = (df['StartDate'] <= date) & (df['EndDate'] >= date)
        
        # cycle through all resvervations, as grouped by the site and the park
        for index, row in df[date_mask].groupby(['Park', 'RegionDescription', 'FacilityID']).nunique().iterrows():
            
            # group the filtered dataframe by unique sit ID, aggregate the columns appropriately
            day_group = df[date_mask].groupby('FacilityID').agg({'StayLen': np.mean, 'BookingHorizon': np.mean,
                                                       'UseFee' : np.mean, 'NumberOfPeople' : np.sum, 
                                                       'DailyRate' : np.sum})
            # assign variables for the daily 
            stay_len = day_group.loc[index[2], 'StayLen']
            book_hor = day_group.loc[index[2], 'BookingHorizon']
            avg_fee = day_group.loc[index[2], 'UseFee']
            num_people = day_group.loc[index[2], 'NumberOfPeople']
            day_revenue = day_group.loc[index[2], 'DailyRate']
            
            # add daily reservation information to the list
            site_list.append([date.date(), index[2], index[0], index[1], row['OrderNumber'], num_people, stay_len, book_hor, avg_fee, day_revenue])
            
        
        count += 1
        bar.update(count)
    
    bar.finish()
    return pd.DataFrame(site_list, columns = ['Date', 'FacilityID', 'Site', 'Park', 'Reservations', 
                                              'NumberOfPeople', 'AvgStayLen', 'AvgBookingHorizon', 'AverageFee', 'DailyRevenue'])

In [70]:
# run site aggregator
# note: this may take a while depending on the size and parameters of the dataframe
rez_18 = nps_site_aggregator(optimized_rez)

[============================================] 100% Time: 0:06:00 Time: 0:06:00


In [71]:
rez_18['Park'].unique()

array(['Assateague Island National Seashore',
       'Grand Canyon National Park', 'Big South Fork National River',
       'Gateway National Recreation Area',
       'Gulf Islands National Seashore',
       'Cape Hatteras National Seashore',
       'Great Smoky Mountains National Park', 'Acadia National Park',
       'Chickasaw National Recreation Area', 'CONGAREE NATIONAL PARK',
       'Yosemite National Park', 'Rocky Mountain National Park',
       'Catoctin National Park', 'Point Reyes National Seashore',
       'Zion National Park', 'Chiricahua National Monument',
       'Cape Lookout National Seashore', 'Greenbelt Park',
       'Obed Wild and Scenic River', 'Colorado National Monument',
       'Channel Islands National Park', 'Shenandoah National Park',
       'Joshua Tree National Park',
       'Chaco Culture National Historic Park', 'Blue Ridge Parkway',
       'Pinnacles National Park',
       'Sleeping Bear Dunes National Lakeshore', 'Catoctin Mountain Park',
       'Lassen Vo

In [72]:
rez_18.to_csv("./data/Aggregated-Data/nps_agg_2017.csv")

In [73]:
rez_18.groupby('Site').mean()

,FacilityID,Reservations,NumberOfPeople,AvgStayLen,AvgBookingHorizon,AverageFee,DailyRevenue
Site,,,,,,,
AKERS,234442.0,2.493506,53.688312,3.073810,62.476840,78.616667,61.090908
ALLEY SPRING,234046.0,23.492611,126.950739,5.279566,110.018161,33.590445,97.241381
ANACAPA ISLAND,232502.0,5.430962,18.669456,2.942152,46.080952,13.071648,36.056485
ANTHONY CREEK HORSE CAMP,232485.0,2.467532,8.240260,4.594108,64.649460,41.162419,23.961039
APGAR GROUP SITES,234669.0,5.726316,77.473684,5.219215,186.838843,139.378900,157.131578
APPALACHIAN CLUBHOUSE,233299.0,1.056604,80.037736,1.367925,246.613208,338.679245,329.433962
ASPENGLEN CAMPGROUND,233187.0,92.560000,295.480000,3.502883,106.682547,42.744651,1218.288005
ASSATEAGUE ISLAND NATIONAL SEASHORE CAMPGROUND,232507.0,142.659824,561.976540,3.945301,79.174058,65.209539,2330.542520
Adirondack Shelters,233321.0,1.823529,5.422460,2.406863,19.922014,0.000000,7.577540
